In [1]:
import numpy as np
import igl
import meshplot as mp

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [13]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Setting up  the constraints

In [7]:
# Add here the code to generate the additional points and constraints

# Function to find the closest point to as point in a set of points
def find_closest_point(point, points):
    distances = np.linalg.norm(points - point, axis=1)
    return np.argmin(distances)

# Function to build constraint equations
def build_constraints(points, normals, epsilon_factor=0.01):
    # Compute the bounding box diagonal
    bbox_min = np.min(points, axis=0)
    bbox_max = np.max(points, axis=0)
    bbox_diag = np.linalg.norm(bbox_max - bbox_min)
    # Compute epsilon
    epsilon = epsilon_factor * bbox_diag
    # Points
    p = np.zeros((3*len(points), 3))
    # Constraints
    f = np.zeros(3*len(points))

    # For each point
    for i, pi in enumerate(points):
        # Add constraint f(pi) = 0
        p[3 * i] = pi
        f[3 * i] = 0.0
        # Compute pi+N and pi-2N
        for j, sign in enumerate([1, -1]):
            epsilon_current = epsilon
            while True:
                # Compute pi_N / pi_-2N
                pi_N = pi + (sign * epsilon_current * normals[i])
                # Find the closest point to pi_N
                closest_index = find_closest_point(pi_N, points)
                # If the point is the same, break
                if np.array_equal(points[closest_index], pi):
                    break
                # Otherwise, update epsilon and repeat
                epsilon_current /= 2
            # Append new point and constraint
            p[3 * i + j + 1] = pi_N
            f[3 * i + j + 1] = sign * epsilon
    
    p = np.array(p)
    f = np.array(f)
    
    return p, f

In [27]:
# Generate the new points and constraints
p, f = build_constraints(pi, ni)

# Create color vector
colors = np.zeros((p.shape[0], 3))
colors[0::3] = [0, 0, 1]
colors[1::3] = [1, 0, 0]
colors[2::3] = [0, 1, 0]
# Visualize the new point cloud 
mp.plot(p, c=colors, shading={"point_size": 6})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

# MLS function

In [ ]:
# This is mockup code, generating the distance field from the surface of the unit sphere in analytic form
# The code is provided to show the desired visualization
# Use tet_grid to generate the grid appropriate to your data
# Generate function fx at all nodes of the grid by evaluating the MLS function

In [28]:
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 10

In [29]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function
center = np.array([0., 0., 0.])
radius = 1
fx = np.linalg.norm(x-center, axis=1) - radius

In [30]:
# Treshold fx to visualize inside outside

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Marching to extract surface

In [31]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…